In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, GPT2Config
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from tqdm import tqdm
import pandas as pd
import wandb

In [2]:
wandb.init(project="gpt2-sentiment-analysis", name='ayesha2')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aamjad (tab-llm-finetuning). Use `wandb login --relogin` to force relogin


In [3]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [4]:
train_df.columns

Index(['review', 'sentiment'], dtype='object')

In [5]:
class MoviewReviewData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [6]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


In [7]:
# Now you can tokenize the data with padding
train_encodings = tokenizer(train_df['review'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_df['review'].tolist(), truncation=True, padding=True, max_length=512)

In [8]:
# Convert labels to numeric
train_labels = train_df['sentiment'].map({'positive': 1, 'negative': 0}).tolist()
test_labels = test_df['sentiment'].map({'positive': 1, 'negative': 0}).tolist()

In [9]:
# Create the dataset
train_dataset = MoviewReviewData(train_encodings, train_labels)
test_dataset = MoviewReviewData(test_encodings, test_labels)


In [10]:
config = GPT2Config.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id, num_labels=2)
model = GPT2ForSequenceClassification.from_pretrained(model_name, config=config)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
print("Loading training arguments...")
training_args = TrainingArguments(
    output_dir='data/result',  # output directory for model and checkpoints
    num_train_epochs=4,           # total number of training epochs
    evaluation_strategy="steps",
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='data/logs',            # directory for storing logs
    logging_steps=50,
    save_steps=1000,
    save_total_limit=2,
    report_to="wandb",                # Disables wandb logging
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Fine-tune the model
print("Start training...")
trainer.train()

print("Saving finetuned gpt2...")
model.save_pretrained('data/result/fine_tuned_gpt2_model')
tokenizer.save_pretrained('data/result/fine_tuned_gpt2_model')

print("Start testing...")
# eval mode on model
trainer.evaluate()
model.eval()

Loading training arguments...
Start training...


Step,Training Loss,Validation Loss
50,1.254600,0.922140
100,1.034700,0.730896
150,0.831000,0.581736
200,0.506700,0.320715
250,0.460700,0.373226
300,0.430000,0.429388
350,0.448100,0.262077
400,0.426100,0.310390
450,0.345200,0.305080
500,0.348200,0.322495


In [ ]:
X_train = train_df['review']
y_train = train_df['sentiment']
X_test = test_df['review']
y_test = test_df['sentiment']

In [ ]:
model = GPT2ForSequenceClassification.from_pretrained('data/result/fine_tuned_gpt2_model')
tokenizer = GPT2Tokenizer.from_pretrained('data/result/fine_tuned_gpt2_model')

In [ ]:
y_train_numeric = y_train.map({'positive': 1, 'negative': 0}).astype(int)
y_test_numeric = y_test.map({'positive': 1, 'negative': 0}).astype(int)

In [ ]:
from torch.utils.data import DataLoader

#test_encodings = tokenizer(test_data['review'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")

# Convert labels to a tensor
labels = torch.tensor(y_test_numeric)

# Create a DataLoader for the test set (optional, for batch processing)
test_dataset = MoviewReviewData(test_encodings, labels)
test_loader = DataLoader(test_dataset, batch_size=16)

def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    correct_predictions = 0
    total_predictions = 0

    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

# Evaluate the model
accuracy = evaluate(model, test_loader)
print(f"Model accuracy on the test set: {accuracy:.2f}%")